# 대한민국 출생아수와 합계출산율 분석

이 노트북에서는 대한민국의 출생아수와 합계출산율 데이터를 분석하고 시각화합니다. 
주요 분석 내용:
1. 데이터 클렌징 및 전처리
2. 시계열 데이터 시각화
3. 통계 분석
4. 상관관계 분석
5. 추세 예측

## 1. 데이터 불러오기 및 라이브러리 설정

필요한 라이브러리를 임포트하고 한글 폰트 설정을 진행합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from datetime import datetime

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.style.use('seaborn')

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

# 데이터 읽기
df = pd.read_excel('출생아수__합계출산율__자연증가_등_20240726084835.xlsx')

## 2. 데이터 클렌징

데이터셋의 기본 정보를 확인하고, 필요한 전처리 작업을 수행합니다:
- 결측치 확인 및 처리
- 데이터 타입 변환
- 불필요한 컬럼 제거
- 이상치 확인 및 처리

In [ ]:
# 데이터 기본 정보 확인
print("데이터셋 기본 정보:")
print(df.info())
print("\n처음 5개 행:")
print(df.head())

# 결측치 확인
print("\n결측치 현황:")
print(df.isnull().sum())

# 데이터 클렌징 함수 정의
def clean_data(df):
    # 복사본 생성
    df_cleaned = df.copy()
    
    # 불필요한 컬럼 제거 (실제 컬럼명에 따라 수정 필요)
    # df_cleaned = df_cleaned.drop(['불필요한_컬럼'], axis=1)
    
    # 결측치 처리
    df_cleaned = df_cleaned.fillna(method='ffill')
    
    # 데이터 타입 변환
    # 연도 컬럼을 정수형으로 변환
    df_cleaned['연도'] = pd.to_numeric(df_cleaned['연도'], errors='coerce')
    
    # 출생아수와 합계출산율을 숫자형으로 변환 (쉼표 제거 후)
    df_cleaned['출생아수'] = pd.to_numeric(df_cleaned['출생아수'].astype(str).str.replace(',', ''), errors='coerce')
    df_cleaned['합계출산율'] = pd.to_numeric(df_cleaned['합계출산율'], errors='coerce')
    
    return df_cleaned

# 데이터 클렌징 수행
df_cleaned = clean_data(df)

print("\n클렌징 후 데이터 정보:")
print(df_cleaned.info())
print("\n기초 통계량:")
print(df_cleaned.describe())

## 3. 시계열 데이터 시각화

연도별 출생아수와 합계출산율의 변화 추이를 다양한 그래프로 시각화합니다.

In [ ]:
# 그래프 크기 설정
plt.figure(figsize=(15, 10))

# 출생아수 추이 그래프
plt.subplot(2, 1, 1)
sns.lineplot(data=df_cleaned, x='연도', y='출생아수', marker='o')
plt.title('연도별 출생아수 추이')
plt.xlabel('연도')
plt.ylabel('출생아수')
plt.grid(True)

# 합계출산율 추이 그래프
plt.subplot(2, 1, 2)
sns.lineplot(data=df_cleaned, x='연도', y='합계출산율', marker='o', color='red')
plt.title('연도별 합계출산율 추이')
plt.xlabel('연도')
plt.ylabel('합계출산율')
plt.grid(True)

plt.tight_layout()
plt.show()

# 연도별 변화율 시각화
df_cleaned['출생아수_변화율'] = df_cleaned['출생아수'].pct_change() * 100
df_cleaned['합계출산율_변화율'] = df_cleaned['합계출산율'].pct_change() * 100

plt.figure(figsize=(15, 6))
plt.plot(df_cleaned['연도'], df_cleaned['출생아수_변화율'], label='출생아수 변화율', marker='o')
plt.plot(df_cleaned['연도'], df_cleaned['합계출산율_변화율'], label='합계출산율 변화율', marker='o')
plt.title('연도별 출생아수와 합계출산율의 변화율')
plt.xlabel('연도')
plt.ylabel('변화율 (%)')
plt.legend()
plt.grid(True)
plt.show()

## 4. 통계 분석 수행

출생아수와 합계출산율에 대한 기본적인 통계 분석을 수행합니다.

In [ ]:
# 기초 통계량 계산
print("출생아수 기초 통계량:")
print(df_cleaned['출생아수'].describe())
print("\n합계출산율 기초 통계량:")
print(df_cleaned['합계출산율'].describe())

# 연도별 변화 분석
print("\n연도별 변화율 통계:")
print("\n출생아수 연간 평균 변화율: {:.2f}%".format(df_cleaned['출생아수_변화율'].mean()))
print("합계출산율 연간 평균 변화율: {:.2f}%".format(df_cleaned['합계출산율_변화율'].mean()))

# 최근 5년 추세
recent_years = df_cleaned.tail(5)
print("\n최근 5년 추세:")
print(recent_years[['연도', '출생아수', '합계출산율', '출생아수_변화율', '합계출산율_변화율']])

## 5. 상관관계 분석

출생아수와 합계출산율 간의 상관관계를 분석합니다.

In [ ]:
# 상관관계 계산
correlation = df_cleaned['출생아수'].corr(df_cleaned['합계출산율'])
print(f"출생아수와 합계출산율의 상관계수: {correlation:.4f}")

# 산점도 그래프
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_cleaned, x='출생아수', y='합계출산율')
plt.title('출생아수와 합계출산율의 상관관계')
plt.xlabel('출생아수')
plt.ylabel('합계출산율')
plt.grid(True)
plt.show()

# 히트맵
plt.figure(figsize=(8, 6))
correlation_matrix = df_cleaned[['출생아수', '합계출산율']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('상관관계 히트맵')
plt.show()

## 6. 추세 예측

간단한 선형 회귀 모델을 사용하여 향후 출생아수와 합계출산율의 추세를 예측합니다.

In [ ]:
# 선형 회귀 모델 생성
def create_prediction_model(df, target_col):
    X = df[['연도']].values
    y = df[target_col].values
    
    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 미래 5년 예측
    future_years = np.array(range(df['연도'].max() + 1, df['연도'].max() + 6)).reshape(-1, 1)
    future_predictions = model.predict(future_years)
    
    return future_years, future_predictions, model.score(X_test, y_test)

# 출생아수 예측
future_years_births, predicted_births, score_births = create_prediction_model(df_cleaned, '출생아수')
print(f"출생아수 예측 모델 R² 점수: {score_births:.4f}")

# 합계출산율 예측
future_years_rate, predicted_rates, score_rates = create_prediction_model(df_cleaned, '합계출산율')
print(f"합계출산율 예측 모델 R² 점수: {score_rates:.4f}")

# 예측 결과 시각화
plt.figure(figsize=(15, 10))

# 출생아수 예측 그래프
plt.subplot(2, 1, 1)
plt.plot(df_cleaned['연도'], df_cleaned['출생아수'], label='실제 출생아수', marker='o')
plt.plot(future_years_births.flatten(), predicted_births, label='예측 출생아수', linestyle='--', marker='o')
plt.title('출생아수 추세 예측')
plt.xlabel('연도')
plt.ylabel('출생아수')
plt.legend()
plt.grid(True)

# 합계출산율 예측 그래프
plt.subplot(2, 1, 2)
plt.plot(df_cleaned['연도'], df_cleaned['합계출산율'], label='실제 합계출산율', marker='o')
plt.plot(future_years_rate.flatten(), predicted_rates, label='예측 합계출산율', linestyle='--', marker='o')
plt.title('합계출산율 추세 예측')
plt.xlabel('연도')
plt.ylabel('합계출산율')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 예측 결과 출력
print("\n향후 5년 예측 결과:")
for i in range(5):
    year = future_years_births.flatten()[i]
    births = predicted_births[i]
    rate = predicted_rates[i]
    print(f"{year}년 예측 - 출생아수: {births:.0f}, 합계출산율: {rate:.2f}")